In this notebook, I have implemented single object tracking using OpenCV

In [1]:
import cv2
import numpy as np

In [3]:
# video path
video1_path = 'videos/walk.mp4'
video2_path = 'videos/walk_cycle.mp4'
video3_path = 'videos/walk_dog.mp4'
video4_path = "videos/football.mp4"
video5_path = "videos/demo.mp4"
video6_path = "videos/byc.mp4"

In [17]:
video = cv2.VideoCapture(video1_path)
height = 400
width = 600

In [7]:
video = cv2.VideoCapture(video3_path)
height = 500
width = 600
bbox = (110, 100, 220, 250)

while True:
    # get the current frame
    check, frame = video.read()
    if not check:
        print(f"Failed to grab the frame, please check you video!!")
        break
    # resized the frame with expected size 
    resized_frame = cv2.resize(frame, (width, height))

    # drawing the boxes 
    # Unpack bbox to get x, y, w, h
    
    x, y, w, h = bbox
    cv2.rectangle(resized_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # showing the video
    cv2.imshow('video', resized_frame)
    
    key = cv2.waitKey(1)
    # press Esc to break 
    if key == 27:
        break
    pass

video.release()
cv2.destroyAllWindows()

In [5]:
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

In [7]:
print(f"Major Version: {major_ver}")
print(f"Minor Version: {minor_ver}")
print(f"Subminor version: {subminor_ver}")

Major Version: 4
Minor Version: 11
Subminor version: 0


In [15]:
import cv2


height = 400
width = 600

# Set up tracker
tracker_types = ['BOOSTING', 'MIL', 'KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
tracker_type = tracker_types[1]  # You can change the tracker type here

# Read the video
video = cv2.VideoCapture(video1_path)
check, frame = video.read()
resized_frame = cv2.resize(frame, (width, height))

if not check:
    print(f"Failed to grab the frame, please check your video!")
    video.release()
    cv2.destroyAllWindows()
    exit()

# Select ROI manually
bbox = cv2.selectROI("Select ROI", resized_frame, fromCenter=False, showCrosshair=True)
cv2.destroyWindow("Select ROI")

# Check if bounding box is valid
if bbox == (0, 0, 0, 0):
    print("No ROI selected, exiting.")
    video.release()
    cv2.destroyAllWindows()
    exit()

# Initialize the tracker based on the selected type
if tracker_type == 'BOOSTING':
    tracker = cv2.TrackerBoosting_create()
elif tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create()
elif tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create()
elif tracker_type == 'TLD':
    tracker = cv2.TrackerTLD_create()
elif tracker_type == 'MEDIANFLOW':
    tracker = cv2.TrackerMedianFlow_create()
elif tracker_type == 'GOTURN':
    tracker = cv2.TrackerGOTURN_create()
elif tracker_type == 'MOSSE':
    tracker = cv2.TrackerMOSSE_create()
elif tracker_type == 'CSRT':
    tracker = cv2.TrackerCSRT_create()

# Initialize tracker with first frame and selected bounding box
tracker.init(resized_frame, bbox)

# To keep track of the object's movement (previous points)
points = []

# Loop through the video frames
while True:
    # Read a new frame
    check, frame = video.read()
    if not check:
        print("Failed to grab the frame, ending video.")
        break

    # Resize the frame
    resized_frame = cv2.resize(frame, (width, height))

    # Update tracker
    ok, bbox = tracker.update(resized_frame)

    # If tracking is successful, draw the bounding box
    if ok:
        p1 = (int(bbox[0]), int(bbox[1]))  # top left corner
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))  # bottom right corner
        # cv2.rectangle(resized_frame, p1, p2, (0, 255, 0), 2)  # draw the bounding box in green

        # Calculate the center of the bounding box
        center = (int(bbox[0] + bbox[2] / 2), int(bbox[1] + bbox[3] / 2))
        
        # Draw a point at the center of the bounding box
        cv2.circle(resized_frame, center, 5, (0, 0, 255), -1)  # red point
        
        # Store the center point for tracking the movement
        points.append(center)
        
        # Draw lines connecting the previous points to visualize movement
        if len(points) > 1:
            for i in range(1, len(points)):
                cv2.line(resized_frame, points[i-1], points[i], (255, 0, 0), 2)  # blue line
    else:
        cv2.putText(resized_frame, "Tracking failure", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

    # Show the frame with bounding box
    cv2.imshow('Tracking', resized_frame)

    # Exit if 'Esc' key is pressed
    key = cv2.waitKey(1)
    if key == 27:
        break

# Release the video capture object and close all windows
video.release()
cv2.destroyAllWindows()


Failed to grab the frame, ending video.
